In [10]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk, simpledialog
from PIL import Image, ImageTk
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from skimage.feature import local_binary_pattern, hog
from skimage import exposure
import os
import threading
import time
import glob
from collections import Counter

class FaceRecognitionGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Face Recognition System")
        self.root.geometry("900x700")
        self.root.configure(bg='#f0f0f0')
        
        # Initialize model and data
        self.knn_model = None
        self.label_map = None
        self.reverse_label_map = None
        self.face_cascade = None
        
        # Camera variables
        self.camera = None
        self.camera_active = False
        self.camera_thread = None
        self.prediction_interval = 0.5  # Predict every 0.5 seconds for performance
        self.last_prediction_time = 0
        self.frame_skip = 2  # Process every 2nd frame for better performance
        self.frame_count = 0
        
        # Load trained model and data
        self.load_model_data()
        
        # Setup GUI
        self.setup_gui()
        
    def load_model_data(self):
        """Load the trained model data and initialize KNN classifier"""
        try:
            # Load feature vectors and labels
            X_train = np.load("X_combined_denoised.npy")
            y_train = np.load("y_combined_denoised.npy")
            self.label_map = np.load("label_map.npy", allow_pickle=True).item()
            
            # Create reverse mapping (label -> person name)
            self.reverse_label_map = {v: k for k, v in self.label_map.items()}
            
            # Initialize and train KNN model
            self.knn_model = KNeighborsClassifier(n_neighbors=3)
            self.knn_model.fit(X_train, y_train)
            
            # Load Haar cascade for face detection
            self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            
            print(f"Model loaded successfully!")
            print(f"Training data shape: {X_train.shape}")
            print(f"Number of people: {len(self.label_map)}")
            print(f"People in dataset: {list(self.label_map.keys())}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load model data: {str(e)}")
            self.root.destroy()
    
    def setup_gui(self):
        """Setup the GUI components"""
        # Title
        title_label = tk.Label(self.root, text="🎯 Face Recognition System", 
                              font=("Arial", 20, "bold"), bg='#f0f0f0', fg='#2c3e50')
        title_label.pack(pady=10)
        
        # Button frame
        button_frame = tk.Frame(self.root, bg='#f0f0f0')
        button_frame.pack(pady=10)
        
        # Upload button
        upload_btn = tk.Button(button_frame, text="📁 Upload Image", 
                              command=self.upload_image,
                              font=("Arial", 12, "bold"),
                              bg='#3498db', fg='white',
                              padx=20, pady=8,
                              cursor='hand2')
        upload_btn.pack(side=tk.LEFT, padx=10)
        
        # Camera button
        self.camera_btn = tk.Button(button_frame, text="📹 Start Camera", 
                                   command=self.toggle_camera,
                                   font=("Arial", 12, "bold"),
                                   bg='#27ae60', fg='white',
                                   padx=20, pady=8,
                                   cursor='hand2')
        self.camera_btn.pack(side=tk.LEFT, padx=10)
        
        # Retrain button
        retrain_btn = tk.Button(button_frame, text="🔄 Retrain Model", 
                               command=self.retrain_model,
                               font=("Arial", 12, "bold"),
                               bg='#9b59b6', fg='white',
                               padx=20, pady=8,
                               cursor='hand2')
        retrain_btn.pack(side=tk.LEFT, padx=10)
        
        # Add Person button
        add_person_btn = tk.Button(button_frame, text="➕ Add New Person", 
                                  command=self.add_new_person,
                                  font=("Arial", 12, "bold"),
                                  bg='#e67e22', fg='white',
                                  padx=20, pady=8,
                                  cursor='hand2')
        add_person_btn.pack(side=tk.LEFT, padx=10)
        
        # Image display frame with fixed dimensions
        self.image_frame = tk.Frame(self.root, bg='#f0f0f0', relief=tk.RAISED, bd=2)
        self.image_frame.pack(pady=10, expand=True, fill=tk.BOTH)
        
        # Image label with proper dimensions for camera feed
        self.image_label = tk.Label(self.image_frame, text="No image selected", 
                                   bg='#ecf0f1', fg='#7f8c8d',
                                   font=("Arial", 12))
        self.image_label.pack(padx=10, pady=10, expand=True)
        
        # Result frame
        result_frame = tk.Frame(self.root, bg='#f0f0f0')
        result_frame.pack(pady=10)
        
        # Prediction label
        tk.Label(result_frame, text="Predicted Person:", 
                font=("Arial", 14, "bold"), bg='#f0f0f0').pack()
        
        self.prediction_label = tk.Label(result_frame, text="No prediction yet", 
                                        font=("Arial", 16, "bold"),
                                        bg='#f0f0f0', fg='#e74c3c')
        self.prediction_label.pack(pady=5)
        
        # Confidence/Distance info
        self.confidence_label = tk.Label(result_frame, text="", 
                                        font=("Arial", 10),
                                        bg='#f0f0f0', fg='#7f8c8d')
        self.confidence_label.pack()
        
        # Status label
        self.status_label = tk.Label(self.root, text="Ready to process images", 
                                    font=("Arial", 10), bg='#f0f0f0', fg='#7f8c8d')
        self.status_label.pack(side=tk.BOTTOM, pady=5)
        
        # Bind window close event
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
    
    def add_new_person(self):
        """Add a new person to the dataset"""
        try:
            # Get person name
            person_name = tk.simpledialog.askstring("Add New Person", 
                                                   "Enter the name of the new person:",
                                                   parent=self.root)
            
            if not person_name:
                return
            
            # Check if person already exists
            if person_name in self.label_map:
                messagebox.showwarning("Person Exists", 
                                     f"Person '{person_name}' already exists in the dataset!")
                return
            
            # Select folder with images for this person
            folder_path = filedialog.askdirectory(
                title=f"Select folder containing images of {person_name}",
                parent=self.root
            )
            
            if not folder_path:
                return
            
            # Process images from the folder
            self.process_new_person_images(person_name, folder_path)
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to add new person: {str(e)}")
    
    def process_new_person_images(self, person_name, folder_path):
        """Process images for a new person and add to dataset"""
        try:
            # Supported image extensions
            extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff', '*.gif']
            
            # Find all image files
            image_files = []
            for ext in extensions:
                image_files.extend(glob.glob(os.path.join(folder_path, ext)))
                image_files.extend(glob.glob(os.path.join(folder_path, ext.upper())))
            
            if len(image_files) == 0:
                messagebox.showwarning("No Images", "No image files found in the selected folder!")
                return
            
            print(f"Processing {len(image_files)} images for {person_name}...")
            
            # Create progress window
            progress_window = tk.Toplevel(self.root)
            progress_window.title("Processing Images")
            progress_window.geometry("400x150")
            progress_window.transient(self.root)
            progress_window.grab_set()
            
            # Progress bar
            progress_label = tk.Label(progress_window, text=f"Processing images for {person_name}...")
            progress_label.pack(pady=10)
            
            progress_bar = ttk.Progressbar(progress_window, length=300, mode='determinate')
            progress_bar.pack(pady=10)
            progress_bar['maximum'] = len(image_files)
            
            # Process images
            new_features = []
            new_labels = []
            processed_count = 0
            
            for i, image_file in enumerate(image_files):
                try:
                    # Update progress
                    progress_bar['value'] = i + 1
                    progress_label.config(text=f"Processing {os.path.basename(image_file)}...")
                    progress_window.update()
                    
                    # Read and process image
                    image = cv2.imread(image_file)
                    if image is None:
                        continue
                    
                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    
                    # Detect faces
                    faces = self.face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(50, 50))
                    
                    if len(faces) == 0:
                        print(f"No face detected in {os.path.basename(image_file)}")
                        continue
                    
                    # Use the largest face
                    largest_face = max(faces, key=lambda x: x[2] * x[3])
                    (x, y, w, h) = largest_face
                    
                    face_roi = gray[y:y+h, x:x+w]
                    face_resized = cv2.resize(face_roi, (100, 100))
                    face_denoised = cv2.GaussianBlur(face_resized, (5, 5), 0)
                    face_equalized = cv2.equalizeHist(face_denoised)
                    
                    # Extract features
                    feature_vector = self.extract_features(face_equalized)
                    
                    new_features.append(feature_vector.flatten())
                    new_labels.append(person_name)
                    processed_count += 1
                    
                except Exception as e:
                    print(f"Error processing {image_file}: {e}")
                    continue
            
            progress_window.destroy()
            
            if processed_count == 0:
                messagebox.showwarning("No Valid Images", 
                                     "No valid face images could be processed!")
                return
            
            # Save new data
            self.save_new_person_data(person_name, new_features, new_labels)
            
            messagebox.showinfo("Success", 
                              f"Successfully added {processed_count} images for {person_name}!\n"
                              f"Click 'Retrain Model' to update the recognition system.")
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to process images: {str(e)}")
    
    def save_new_person_data(self, person_name, new_features, new_labels):
        """Save new person data to files"""
        try:
            # Load existing data
            X_existing = np.load("X_combined_denoised.npy")
            y_existing = np.load("y_combined_denoised.npy")
            
            # Convert new features to numpy array
            X_new = np.array(new_features)
            
            # Add new person to label map
            new_label = max(self.label_map.values()) + 1
            self.label_map[person_name] = new_label
            self.reverse_label_map[new_label] = person_name
            
            # Convert new labels to numeric
            y_new = np.array([new_label] * len(new_labels))
            
            # Combine with existing data
            X_combined = np.vstack([X_existing, X_new])
            y_combined = np.concatenate([y_existing, y_new])
            
            # Save updated data
            np.save("X_combined_denoised.npy", X_combined)
            np.save("y_combined_denoised.npy", y_combined)
            np.save("label_map.npy", self.label_map)
            
            print(f"Saved data for {person_name}: {len(new_features)} samples")
            print(f"Updated dataset shape: {X_combined.shape}")
            
        except Exception as e:
            raise Exception(f"Failed to save new person data: {str(e)}")
    
    def retrain_model(self):
        """Retrain the model with updated dataset"""
        try:
            self.status_label.config(text="Retraining model...")
            self.root.update()
            
            # Show progress dialog
            progress_window = tk.Toplevel(self.root)
            progress_window.title("Retraining Model")
            progress_window.geometry("300x100")
            progress_window.transient(self.root)
            progress_window.grab_set()
            
            progress_label = tk.Label(progress_window, text="Retraining model, please wait...")
            progress_label.pack(pady=20)
            
            progress_bar = ttk.Progressbar(progress_window, mode='indeterminate')
            progress_bar.pack(pady=10)
            progress_bar.start()
            
            progress_window.update()
            
            # Load updated data
            X_train = np.load("X_combined_denoised.npy")
            y_train = np.load("y_combined_denoised.npy")
            self.label_map = np.load("label_map.npy", allow_pickle=True).item()
            
            # Update reverse mapping
            self.reverse_label_map = {v: k for k, v in self.label_map.items()}
            
            # Retrain KNN model
            self.knn_model = KNeighborsClassifier(n_neighbors=3)
            self.knn_model.fit(X_train, y_train)
            
            progress_window.destroy()
            
            # Show updated info
            messagebox.showinfo("Model Retrained", 
                              f"Model successfully retrained!\n\n"
                              f"Dataset size: {X_train.shape[0]} samples\n"
                              f"Number of people: {len(self.label_map)}\n"
                              f"People: {', '.join(sorted(self.label_map.keys()))}")
            
            self.status_label.config(text="Model retrained successfully")
            
            print(f"Model retrained with {X_train.shape[0]} samples")
            print(f"People in dataset: {list(self.label_map.keys())}")
            
        except Exception as e:
            if 'progress_window' in locals():
                progress_window.destroy()
            messagebox.showerror("Retraining Error", f"Failed to retrain model: {str(e)}")
            self.status_label.config(text="Retraining failed")
    
    def toggle_camera(self):
        """Toggle camera on/off"""
        if not self.camera_active:
            self.start_camera()
        else:
            self.stop_camera()
    
    def start_camera(self):
        """Start the camera feed"""
        try:
            # Initialize camera with lower resolution for better performance
            self.camera = cv2.VideoCapture(0)
            
            if not self.camera.isOpened():
                messagebox.showerror("Camera Error", "Could not access camera. Please check if it's connected.")
                return
            
            # Set camera properties for better performance
            self.camera.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            self.camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
            self.camera.set(cv2.CAP_PROP_FPS, 30)
            
            self.camera_active = True
            self.camera_btn.config(text="📹 Stop Camera", bg='#e74c3c')
            self.status_label.config(text="Camera started - Real-time face recognition active")
            
            # Start camera thread
            self.camera_thread = threading.Thread(target=self.camera_loop, daemon=True)
            self.camera_thread.start()
            
        except Exception as e:
            messagebox.showerror("Camera Error", f"Failed to start camera: {str(e)}")
    
    def stop_camera(self):
        """Stop the camera feed"""
        self.camera_active = False
        
        if self.camera:
            self.camera.release()
            self.camera = None
        
        self.camera_btn.config(text="📹 Start Camera", bg='#27ae60')
        self.status_label.config(text="Camera stopped")
        
        # Clear the image display
        self.image_label.config(image="", text="Camera stopped")
        self.image_label.image = None
    
    def camera_loop(self):
        """Main camera loop running in separate thread"""
        while self.camera_active and self.camera:
            try:
                ret, frame = self.camera.read()
                if not ret:
                    break
                
                self.frame_count += 1
                
                # Flip frame horizontally for mirror effect
                frame = cv2.flip(frame, 1)
                
                # Convert BGR to RGB for display
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Process frame for face detection and recognition (skip frames for performance)
                if self.frame_count % self.frame_skip == 0:
                    self.process_camera_frame(frame.copy())
                
                # Update GUI with current frame
                self.update_camera_display(frame_rgb)
                
                # Small delay to prevent overwhelming the GUI
                time.sleep(0.03)  # ~30 FPS
                
            except Exception as e:
                print(f"Camera loop error: {e}")
                break
        
        # Cleanup
        if self.camera:
            self.camera.release()
        self.camera_active = False
    
    def process_camera_frame(self, frame):
        """Process camera frame for face recognition"""
        try:
            current_time = time.time()
            
            # Only make predictions at specified intervals
            if current_time - self.last_prediction_time < self.prediction_interval:
                return
            
            # Convert to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Detect faces (use smaller scale factor for faster detection)
            faces = self.face_cascade.detectMultiScale(gray, 1.2, 4, minSize=(50, 50))
            
            if len(faces) > 0:
                # Use the largest face
                largest_face = max(faces, key=lambda x: x[2] * x[3])
                (x, y, w, h) = largest_face
                
                # Extract face ROI
                face_roi = gray[y:y+h, x:x+w]
                
                # Resize to 100x100
                face_resized = cv2.resize(face_roi, (100, 100))
                
                # Apply Gaussian blur for noise reduction
                face_denoised = cv2.GaussianBlur(face_resized, (5, 5), 0)
                
                # Extract features and make prediction
                feature_vector = self.extract_features(face_denoised)
                self.make_realtime_prediction(feature_vector)
                
                self.last_prediction_time = current_time
            else:
                # No face detected
                self.root.after(0, lambda: self.prediction_label.config(text="No face detected", fg='#e74c3c'))
                self.root.after(0, lambda: self.confidence_label.config(text=""))
                
        except Exception as e:
            print(f"Frame processing error: {e}")
    
    def update_camera_display(self, frame_rgb):
        """Update the camera display in the GUI"""
        try:
            # Get current window size for dynamic resizing
            window_width = self.root.winfo_width()
            window_height = self.root.winfo_height()
            
            # Calculate available space for image (considering other UI elements)
            available_width = max(window_width - 100, 400)  # Min 400px width
            available_height = max(window_height - 300, 300)  # Min 300px height, reserve space for UI
            
            # Get original frame dimensions
            height, width = frame_rgb.shape[:2]
            
            # Calculate scaling to fit within available space while maintaining aspect ratio
            scale_w = available_width / width
            scale_h = available_height / height
            scale = min(scale_w, scale_h, 1.0)  # Don't upscale beyond original size
            
            # Calculate new dimensions
            new_width = int(width * scale)
            new_height = int(height * scale)
            
            # Resize frame
            frame_resized = cv2.resize(frame_rgb, (new_width, new_height))
            
            # Convert to PIL Image and then to PhotoImage
            pil_image = Image.fromarray(frame_resized)
            photo = ImageTk.PhotoImage(pil_image)
            
            # Update GUI in main thread
            self.root.after(0, self._update_image_label, photo, new_width, new_height)
            
        except Exception as e:
            print(f"Display update error: {e}")
    
    def _update_image_label(self, photo, width=None, height=None):
        """Update image label in main thread"""
        if width and height:
            self.image_label.config(image=photo, text="", width=width, height=height)
        else:
            self.image_label.config(image=photo, text="")
        self.image_label.image = photo  # Keep a reference
    
    def make_realtime_prediction(self, feature_vector):
        """Make prediction for real-time camera feed"""
        try:
            # Get prediction
            prediction = self.knn_model.predict(feature_vector)[0]
            
            # Get distances to nearest neighbors
            distances, indices = self.knn_model.kneighbors(feature_vector)
            
            # Convert label to person name
            person_name = self.reverse_label_map.get(prediction, f"Unknown_{prediction}")
            
            # Calculate confidence (inverse of distance)
            confidence = 1 / (1 + distances[0][0])
            confidence_text = f"Confidence: {confidence:.2f} | Distance: {distances[0][0]:.2f}"
            
            # Update GUI in main thread
            self.root.after(0, lambda: self.prediction_label.config(text=person_name, fg='#27ae60'))
            self.root.after(0, lambda: self.confidence_label.config(text=confidence_text))
            
        except Exception as e:
            print(f"Prediction error: {e}")
    
    def upload_image(self):
        """Handle image upload and processing"""
        file_path = filedialog.askopenfilename(
            title="Select an image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff *.gif")]
        )
        
        if file_path:
            # Stop camera if active
            if self.camera_active:
                self.stop_camera()
            
            self.process_image(file_path)
    
    def process_image(self, file_path):
        """Process the uploaded image and make prediction"""
        try:
            self.status_label.config(text="Processing image...")
            self.root.update()
            
            print(f"\n=== PROCESSING IMAGE: {os.path.basename(file_path)} ===")
            
            # Read image with OpenCV
            image = cv2.imread(file_path)
            if image is None:
                raise ValueError("Could not load image")
            
            print(f"Original image shape: {image.shape}")
            
            # Convert to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Detect faces with multiple parameters for better detection
            faces1 = self.face_cascade.detectMultiScale(gray, 1.1, 4, minSize=(50, 50))
            faces2 = self.face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(30, 30))
            
            # Combine results and remove duplicates
            all_faces = list(faces1) + list(faces2)
            if len(all_faces) == 0:
                faces = []
            else:
                # Remove duplicate detections
                faces = []
                for face in all_faces:
                    is_duplicate = False
                    for existing_face in faces:
                        # Check if faces overlap significantly
                        overlap = self.calculate_overlap(face, existing_face)
                        if overlap > 0.5:
                            is_duplicate = True
                            break
                    if not is_duplicate:
                        faces.append(face)
            
            print(f"Faces detected: {len(faces)}")
            
            if len(faces) == 0:
                messagebox.showwarning("No Face Detected", 
                                     "No face was detected in the image. Please try another image with a clearer face.")
                self.status_label.config(text="No face detected")
                self.display_image(file_path)
                return
            
            # Display the original image first
            self.display_image(file_path)
            
            # Use the largest detected face
            largest_face = max(faces, key=lambda x: x[2] * x[3])
            (x, y, w, h) = largest_face
            
            print(f"Selected face region: x={x}, y={y}, w={w}, h={h}")
            
            face_roi = gray[y:y+h, x:x+w]
            
            # Resize to 100x100 (same as training)
            face_resized = cv2.resize(face_roi, (100, 100))
            
            # Apply same preprocessing as training data
            face_denoised = cv2.GaussianBlur(face_resized, (5, 5), 0)
            
            # Optional: Apply histogram equalization for better contrast
            face_equalized = cv2.equalizeHist(face_denoised)
            
            print(f"Preprocessed face shape: {face_equalized.shape}")
            
            # Extract features
            feature_vector = self.extract_features(face_equalized)
            
            # Make prediction
            self.make_prediction(feature_vector)
            
            self.status_label.config(text="Prediction completed successfully")
            
        except Exception as e:
            print(f"Error processing image: {str(e)}")
            messagebox.showerror("Error", f"Failed to process image: {str(e)}")
            self.status_label.config(text="Error processing image")
            try:
                self.display_image(file_path)
            except:
                pass
    
    def calculate_overlap(self, face1, face2):
        """Calculate overlap ratio between two face rectangles"""
        x1, y1, w1, h1 = face1
        x2, y2, w2, h2 = face2
        
        # Calculate intersection
        x_left = max(x1, x2)
        y_top = max(y1, y2)
        x_right = min(x1 + w1, x2 + w2)
        y_bottom = min(y1 + h1, y2 + h2)
        
        if x_right < x_left or y_bottom < y_top:
            return 0.0
        
        intersection = (x_right - x_left) * (y_bottom - y_top)
        area1 = w1 * h1
        area2 = w2 * h2
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0.0
    
    def display_image(self, file_path):
        """Display the uploaded image in the GUI"""
        try:
            # Open image
            pil_image = Image.open(file_path)
            
            # Get current window size for dynamic resizing
            window_width = self.root.winfo_width()
            window_height = self.root.winfo_height()
            
            # Calculate available space for image
            available_width = max(window_width - 100, 400)
            available_height = max(window_height - 300, 300)
            
            # Get original dimensions
            orig_width, orig_height = pil_image.size
            
            # Calculate scaling factor to fit within available space
            scale_w = available_width / orig_width
            scale_h = available_height / orig_height
            scale = min(scale_w, scale_h, 1.0)  # Don't upscale beyond original
            
            # Calculate new dimensions
            new_width = int(orig_width * scale)
            new_height = int(orig_height * scale)
            
            # Resize image
            pil_image = pil_image.resize((new_width, new_height), Image.Resampling.LANCZOS)
            
            # Convert to PhotoImage
            photo = ImageTk.PhotoImage(pil_image)
            
            # Update image label
            self.image_label.config(image=photo, text="", width=new_width, height=new_height)
            self.image_label.image = photo  # Keep a reference
            
        except Exception as e:
            print(f"Error displaying image: {e}")
            self.image_label.config(text="Failed to display image", image="")
    
    def extract_features(self, face_image):
        """Extract LBP and HOG features from the face image"""
        # LBP feature extraction
        lbp = local_binary_pattern(face_image, P=8, R=1, method='uniform')
        lbp_hist, _ = np.histogram(lbp.ravel(), bins=10, range=(0, 10), density=True)
        
        # HOG feature extraction
        hog_features = hog(face_image, 
                          orientations=9,
                          pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2),
                          block_norm='L2-Hys',
                          feature_vector=True)
        
        # Combine features
        combined_features = np.concatenate([lbp_hist, hog_features])
        
        return combined_features.reshape(1, -1)
    
    def make_prediction(self, feature_vector):
        """Make prediction using the trained KNN model with improved logic"""
        try:
            # Get distances to all neighbors (increase k for better analysis)
            k_neighbors = min(9, len(self.knn_model._y))  # Use up to 9 neighbors
            distances, indices = self.knn_model.kneighbors(feature_vector, n_neighbors=k_neighbors)
            
            # Get the labels of nearest neighbors
            neighbor_labels = [self.knn_model._y[idx] for idx in indices[0]]
            neighbor_distances = distances[0]
            
            # Count votes for each person
            label_votes = Counter(neighbor_labels)
            
            # Get the most voted label
            predicted_label = label_votes.most_common(1)[0][0]
            vote_count = label_votes.most_common(1)[0][1]
            
            # Calculate confidence metrics
            min_distance = neighbor_distances[0]
            avg_distance_top3 = np.mean(neighbor_distances[:3])
            
            # Convert label to person name
            person_name = self.reverse_label_map.get(predicted_label, f"Unknown_{predicted_label}")
            
            # Determine if prediction is reliable
            confidence_score = vote_count / min(k_neighbors, 5)  # Voting confidence
            distance_confidence = 1 / (1 + min_distance)  # Distance-based confidence
            
            # Set distance threshold for reliable predictions
            DISTANCE_THRESHOLD = 12.0  # Adjust based on your data
            is_reliable = min_distance < DISTANCE_THRESHOLD and confidence_score >= 0.4
            
            # Update GUI based on reliability
            if is_reliable:
                self.prediction_label.config(text=person_name, fg='#27ae60')
                status_color = '#27ae60'
                status = "Confident"
            else:
                self.prediction_label.config(text=f"{person_name} (?)", fg='#f39c12')
                status_color = '#f39c12'
                status = "Uncertain"
            
            # Show detailed confidence info
            confidence_text = (f"Distance: {min_distance:.2f} | Votes: {vote_count}/{k_neighbors} | "
                             f"Status: {status}")
            self.confidence_label.config(text=confidence_text, fg=status_color)
            
            # Print detailed analysis
            print(f"\n=== PREDICTION ANALYSIS ===")
            print(f"Predicted Person: {person_name}")
            print(f"Minimum Distance: {min_distance:.2f}")
            print(f"Average Distance (top-3): {avg_distance_top3:.2f}")
            print(f"Vote Distribution: {dict(label_votes)}")
            print(f"Confidence Score: {confidence_score:.2f}")
            print(f"Reliability: {status}")
            
            print(f"\nTop-{min(5, k_neighbors)} Nearest Neighbors:")
            for i in range(min(5, len(neighbor_distances))):
                neighbor_label = neighbor_labels[i]
                neighbor_name = self.reverse_label_map.get(neighbor_label, f"Unknown_{neighbor_label}")
                distance = neighbor_distances[i]
                print(f"{i+1}. {neighbor_name}: {distance:.2f}")
            
            # Warning for potentially incorrect predictions
            if not is_reliable:
                print(f"⚠️  WARNING: Prediction may be unreliable!")
                print(f"   - Distance {min_distance:.2f} > threshold {DISTANCE_THRESHOLD}")
                print(f"   - Consider adding more training data or checking image quality")
            
        except Exception as e:
            messagebox.showerror("Prediction Error", f"Failed to make prediction: {str(e)}")
            self.prediction_label.config(text="Prediction failed", fg='#e74c3c')
            self.confidence_label.config(text="")
    
    def on_closing(self):
        """Handle window closing event"""
        if self.camera_active:
            self.stop_camera()
        self.root.destroy()

def main():
    """Main function to run the GUI"""
    # Check if required files exist
    required_files = ["X_combined_denoised.npy", "y_combined_denoised.npy", "label_map.npy"]
    missing_files = [f for f in required_files if not os.path.exists(f)]
    
    if missing_files:
        print(f"Error: Missing required files: {missing_files}")
        print("Please ensure all model files are in the same directory as this script.")
        return
    
    # Create and run GUI
    root = tk.Tk()
    app = FaceRecognitionGUI(root)
    
    try:
        root.mainloop()
    except KeyboardInterrupt:
        print("\nGUI closed by user.")
    except Exception as e:
        print(f"Unexpected error: {e}")
    finally:
        if hasattr(app, 'camera_active') and app.camera_active:
            app.stop_camera()
        root.quit()

if __name__ == "__main__":
    main()

Model loaded successfully!
Training data shape: (3230, 4366)
Number of people: 11
People in dataset: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']

=== PROCESSING IMAGE: frame_00001.jpg ===
Original image shape: (1280, 720, 3)
Faces detected: 1
Selected face region: x=264, y=297, w=216, h=216
Preprocessed face shape: (100, 100)

=== PREDICTION ANALYSIS ===
Predicted Person: 04
Minimum Distance: 2.49
Average Distance (top-3): 2.70
Vote Distribution: {np.int64(3): 9}
Confidence Score: 1.80
Reliability: Confident

Top-5 Nearest Neighbors:
1. 04: 2.49
2. 04: 2.78
3. 04: 2.82
4. 04: 3.00
5. 04: 3.07
